# Conditional GAN

Used to generate new training data for the ransomware families to overcome the skewed distribution of training data towards the benign samples

In [16]:
# Packages
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import cv2

**Change parameters**

-----

In [71]:
# Batch size
batch_size = 64

# Color mode
ch = 'grayscale'

# Image size
iw, ih = 64,64
im_size = (iw,ih)

# Latent dim size
latent_dim = 128

# Number of Epochs
epoch_t = 1000 # Add this point this model has run 2000 epochs through using the checkpoints (see ckpt_cgan_v009)

# Computation environment: Kaggle (0) or Local (1)
cenv = 1


-----------

Automatic notebook preparation

In [4]:
if(ch == 'rgb'):
    chnum = 3
elif(ch == 'grayscale'):
    chnum = 1

Create new folder to save the output of this model

In [5]:
if cenv == 1:
    file_exists = []
    vnum = 1
    dir = "C:/Users/Max/Documents/GitHub/conditional_gan"
    for files in os.listdir(dir):
        if "cgan" in files:
            try:
                vnum = max(vnum, int(files[-3:]))
            except: 
                continue
            new_vnum = vnum + 1
            file_exists.append(True)
        else: 
            file_exists.append(False)
    # If this is the first notebook you want to save, a new folder will be created with version #001
    if sum(file_exists) == 0:
        new_vnum = 1
        print("No matches found")

    else: 
        print(f"{sum(file_exists)} matches(es) found")
        print("--------------")

    # Print new folder name
    print(f"New folder name: cgan-one-class-local-v{new_vnum:03}")
    print("--------------")
    
    # Create new folder with the name of the notebook and the version number
    new_dir = f"C://Users/Max/Documents/GitHub/conditional_gan/cgan-one-class-local-v{new_vnum:03}"
    os.makedirs(new_dir)

12 matches(es) found
--------------
New folder name: cgan-one-class-local-v016
--------------


**Data preprocessing**

In [102]:
# Generate for Occamy.C
if cenv == 1:
    path_root = "C:/Users/Max/Documents/image_data/data_wo_benign/Occamy.C/"
    path_save_imgs = f"C:/Users/Max/Documents/image_data/cgan-one-class-local-v{new_vnum:03}"

In [7]:
datagen = ImageDataGenerator(
    rescale = 1/255 # Pixel values need to be scaled between 0 and 1
)

In [50]:
imgs = np.array([])
for i, file in enumerate(os.listdir(path_root)):
    image_loc = os.path.join(path_root,file)
    
    img = cv2.imread(image_loc,0)
    img = cv2.resize(img, (64,64))
    img = np.array(img)
    img = np.reshape(img,(64,64,1))
    
    imgs = np.append(imgs, img)
    
imgs = imgs.reshape(int(len(imgs)/(64*64)),64,64,1)
    

In [74]:
labels = np.full(imgs.shape[0], 0)
labels = to_categorical(labels)

In [89]:
imgs = imgs.astype('float32')
imgs = imgs / 255.0

In [90]:
num_samples = imgs.shape[0]
num_classes = 1

Create tf.data.Dataset

In [91]:
dataset = tf.data.Dataset.from_tensor_slices((imgs, labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

Calculate number of input channel for Gen and Disc

In [92]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = chnum + num_classes
print(generator_in_channels, discriminator_in_channels)

129 2


# Creating discriminator and generator

In [93]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((iw, ih, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)


In [94]:
# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(8 * 8 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((8, 8, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [95]:
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 64)        1216      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 256)         295168    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 8, 8, 256)         0         
_________________________________________________________________
global_max_pooling2d_2 (Glob (None, 256)             

In [96]:
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 8256)              1073280   
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 8256)              0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 8, 8, 129)         0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 16, 16, 128)       264320    
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 32, 32, 128)       262272    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 32, 32, 128)       0 

**Create Conditional GAN**

In [97]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[ih * iw]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, iw, ih, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

**Optimizers**

In [98]:
# Define optimizers
d_optimizer=keras.optimizers.Adam(learning_rate=0.0003)
g_optimizer=keras.optimizers.Adam(learning_rate=0.0003)

**Checkpoints**

In [99]:
class GANMonitor(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        
        # Save the model every 5 epochs 
        if (epoch + 1) % 10 == 0:
          checkpoint.save(file_prefix = checkpoint_prefix)

In [100]:
if cenv == 0:
    checkpoint_dir = '/kaggle/working/checkpoints'
if cenv == 1:
    checkpoint_dir = f'{new_dir}'
    
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=g_optimizer,
                                 discriminator_optimizer=d_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Training C-GAN

In [101]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=epoch_t, 
        callbacks=GANMonitor()
)


Epoch 1/1000
14/14 [==============================] - 4s 117ms/step - g_loss: 0.7088 - d_loss: 0.6799
Epoch 2/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 0.7626 - d_loss: 0.6044
Epoch 3/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 1.8176 - d_loss: 0.2664
Epoch 4/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 0.7438 - d_loss: 0.8496
Epoch 5/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.7498 - d_loss: 0.9585
Epoch 6/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 1.9317 - d_loss: 0.2371
Epoch 7/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 1.4550 - d_loss: 0.8361
Epoch 8/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 0.3681 - d_loss: 1.1727
Epoch 9/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 1.5980 - d_loss: 0.3182
Epoch 10/1000
14/14 [==============================] - 1s 101ms/step - g_loss: 1.7

14/14 [==============================] - 1s 103ms/step - g_loss: 1.9475 - d_loss: 0.3949
Epoch 81/1000
14/14 [==============================] - 1s 105ms/step - g_loss: 0.7844 - d_loss: 0.7999
Epoch 82/1000
14/14 [==============================] - 1s 105ms/step - g_loss: 1.6486 - d_loss: 0.2826
Epoch 83/1000
14/14 [==============================] - 2s 112ms/step - g_loss: 2.3308 - d_loss: 0.1618
Epoch 84/1000
14/14 [==============================] - 1s 105ms/step - g_loss: 1.6993 - d_loss: 0.5797
Epoch 85/1000
14/14 [==============================] - 2s 110ms/step - g_loss: 0.3760 - d_loss: 1.2660
Epoch 86/1000
14/14 [==============================] - 2s 108ms/step - g_loss: 0.9402 - d_loss: 0.6100
Epoch 87/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.9237 - d_loss: 0.5441
Epoch 88/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.9491 - d_loss: 0.5856
Epoch 89/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.2419 -

14/14 [==============================] - 1s 104ms/step - g_loss: 4.2265 - d_loss: 0.0255
Epoch 160/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 2.4312 - d_loss: 0.3880
Epoch 161/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 1.1375 - d_loss: 0.7790
Epoch 162/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.9647 - d_loss: 0.7993
Epoch 163/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 2.0908 - d_loss: 0.2140
Epoch 164/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 2.0290 - d_loss: 0.2375
Epoch 165/1000
14/14 [==============================] - 6s 486ms/step - g_loss: 2.1231 - d_loss: 0.3023
Epoch 166/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.9798 - d_loss: 0.6820
Epoch 167/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.9935 - d_loss: 0.4654
Epoch 168/1000
14/14 [==============================] - 1s 103ms/step - g_loss:

14/14 [==============================] - 1s 103ms/step - g_loss: 1.1500 - d_loss: 0.5153
Epoch 238/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.7177 - d_loss: 0.8219
Epoch 239/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.6133 - d_loss: 0.8089
Epoch 240/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.5864 - d_loss: 0.8180
Epoch 241/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.5363 - d_loss: 0.8543
Epoch 242/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.5687 - d_loss: 0.7870
Epoch 243/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.6923 - d_loss: 0.6844
Epoch 244/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.8105 - d_loss: 0.6090
Epoch 245/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.9453 - d_loss: 0.6025
Epoch 246/1000
14/14 [==============================] - 2s 108ms/step - g_loss:

14/14 [==============================] - 2s 108ms/step - g_loss: 3.7427 - d_loss: 0.0305
Epoch 316/1000
14/14 [==============================] - 2s 107ms/step - g_loss: 3.0958 - d_loss: 0.1347
Epoch 317/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 3.5531 - d_loss: 0.0485
Epoch 318/1000
14/14 [==============================] - 1s 105ms/step - g_loss: 1.5348 - d_loss: 0.6972
Epoch 319/1000
14/14 [==============================] - 2s 115ms/step - g_loss: 2.1013 - d_loss: 0.1979
Epoch 320/1000
14/14 [==============================] - 2s 127ms/step - g_loss: 3.0391 - d_loss: 0.1301
Epoch 321/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 2.7526 - d_loss: 0.1700
Epoch 322/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 2.4902 - d_loss: 0.8345
Epoch 323/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 1.5509 - d_loss: 0.8178
Epoch 324/1000
14/14 [==============================] - 2s 110ms/step - g_loss:

...

14/14 [==============================] - 1s 104ms/step - g_loss: 2.9848 - d_loss: 0.1318
Epoch 628/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 3.3485 - d_loss: 0.2825
Epoch 629/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 4.9546 - d_loss: 0.0802
Epoch 630/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.4921 - d_loss: 1.1600
Epoch 631/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.9911 - d_loss: 0.5782
Epoch 632/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.5831 - d_loss: 0.8210
Epoch 633/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.6398 - d_loss: 0.7619
Epoch 634/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.7037 - d_loss: 0.6905
Epoch 635/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.8046 - d_loss: 0.6271
Epoch 636/1000
14/14 [==============================] - 1s 104ms/step - g_loss:

14/14 [==============================] - 1s 103ms/step - g_loss: 0.7566 - d_loss: 0.7212
Epoch 706/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 1.0842 - d_loss: 0.4935
Epoch 707/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.2328 - d_loss: 0.5476
Epoch 708/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.6961 - d_loss: 0.9151
Epoch 709/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.0726 - d_loss: 0.5138
Epoch 710/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 1.0830 - d_loss: 0.5045
Epoch 711/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.7543 - d_loss: 0.7394
Epoch 712/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.6929 - d_loss: 0.7948
Epoch 713/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.6432 - d_loss: 0.8747
Epoch 714/1000
14/14 [==============================] - 1s 103ms/step - g_loss:

14/14 [==============================] - 2s 113ms/step - g_loss: 1.3029 - d_loss: 0.5244
Epoch 784/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.7319 - d_loss: 0.8147
Epoch 785/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.5762 - d_loss: 1.0302
Epoch 786/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.7545 - d_loss: 0.2780
Epoch 787/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.2286 - d_loss: 0.6533
Epoch 788/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.6083 - d_loss: 1.0328
Epoch 789/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 0.7315 - d_loss: 0.8493
Epoch 790/1000
14/14 [==============================] - 1s 103ms/step - g_loss: 1.1275 - d_loss: 0.6720
Epoch 791/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 0.7615 - d_loss: 0.9374
Epoch 792/1000
14/14 [==============================] - 1s 103ms/step - g_loss:

14/14 [==============================] - 2s 107ms/step - g_loss: 0.8585 - d_loss: 0.6014
Epoch 862/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 1.0426 - d_loss: 0.4998
Epoch 863/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 1.1690 - d_loss: 0.4538
Epoch 864/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 0.6961 - d_loss: 0.8689
Epoch 865/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 0.7353 - d_loss: 0.8221
Epoch 866/1000
14/14 [==============================] - 1s 107ms/step - g_loss: 0.8193 - d_loss: 0.7356
Epoch 867/1000
14/14 [==============================] - 1s 105ms/step - g_loss: 0.7582 - d_loss: 0.7123
Epoch 868/1000
14/14 [==============================] - 2s 108ms/step - g_loss: 0.8388 - d_loss: 0.6061
Epoch 869/1000
14/14 [==============================] - 1s 104ms/step - g_loss: 0.8402 - d_loss: 0.6310
Epoch 870/1000
14/14 [==============================] - 1s 103ms/step - g_loss:

14/14 [==============================] - 2s 107ms/step - g_loss: 0.7409 - d_loss: 0.6877
Epoch 940/1000
14/14 [==============================] - 2s 113ms/step - g_loss: 0.7830 - d_loss: 0.6616
Epoch 941/1000
14/14 [==============================] - 2s 114ms/step - g_loss: 0.7879 - d_loss: 0.6387
Epoch 942/1000
14/14 [==============================] - 2s 119ms/step - g_loss: 0.8592 - d_loss: 0.6043
Epoch 943/1000
14/14 [==============================] - 2s 111ms/step - g_loss: 0.8628 - d_loss: 0.6386
Epoch 944/1000
14/14 [==============================] - 1s 106ms/step - g_loss: 0.9023 - d_loss: 0.6635
Epoch 945/1000
14/14 [==============================] - 2s 107ms/step - g_loss: 0.9829 - d_loss: 0.6265
Epoch 946/1000
14/14 [==============================] - 2s 109ms/step - g_loss: 0.7465 - d_loss: 0.7035
Epoch 947/1000
14/14 [==============================] - 2s 108ms/step - g_loss: 0.7370 - d_loss: 0.6793
Epoch 948/1000
14/14 [==============================] - 2s 109ms/step - g_loss:

# Create new training images using the Conditional GAN

In [103]:
# We first extract the trained generator from our Conditiona GAN.
trained_gen = cond_gan.generator

# Number of images that are generated per class
num_interpolation = 15000  # @param {type:"integer"}

# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))


def interpolate_class(first_number, second_number):
    # Convert the start and end labels to one-hot encoded vectors.
    first_label = keras.utils.to_categorical([first_number], num_classes)
    second_label = keras.utils.to_categorical([second_number], num_classes)
    first_label = tf.cast(first_label, tf.float32)
    second_label = tf.cast(second_label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    percent_second_label = tf.linspace(0, 1, num_interpolation)[:, None]
    percent_second_label = tf.cast(percent_second_label, tf.float32)
    interpolation_labels = (
        first_label * (1 - percent_second_label) + second_label * percent_second_label
    )

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake



In [104]:
# Create new directory for saving folder
os.makedirs(path_save_imgs)

In [121]:
# Retrieve class name based on number
classes_list = path_root[path_root.find("data_wo_benign") + len("data_wo_benign") + 1:-1]
classes_list


'Occamy.C'

In [123]:
# Create images for every class and store in seperate folder
for i in range(num_classes):
    class_name = classes_list
    class_dir = f"{path_save_imgs}/{class_name}"
    os.makedirs(class_dir)
    start_class = i
    end_class = i
    fake_images = interpolate_class(start_class, end_class)
    fake_images *= 255
    converted_images = fake_images.astype(np.uint8)
    converted_images = tf.image.resize(converted_images, (64, 64)).numpy().astype(np.uint8)
    for j in range(num_interpolation):
        np_array = np.squeeze(converted_images[j], axis=2)
        im = Image.fromarray((np_array))
        im.save(f"{class_dir}/gen_imgs_{class_name}_{j}.png")    